<a href="https://colab.research.google.com/github/ahmed192a/ImageProcessing-PatternRecognition/blob/main/P1.Advanced-Lane-Detection/Phase1_Advanced_Lane_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Phase one: Lane Detection
**In this first phase**, the goal is to write a software pipeline to identify the lane boundaries in a video from a front-facing camera on a car. it’s required to find and track the lane lines and the
position of the car from the center of the lane. <br>
As a bonus, track the radius of curvature of the road too.

Assume the camera is mounted at the center of the car, such that the lane center is the midpoint at the bottom of the image between the two lines you've detected. 

The offset of the lane center from the center of the image (converted from pixels to meters) is your distance from the center of the lane.

# Imports

# Line Class


# Prespective Transform Class


# Globale Var


# Blend Frames


# Threshold methods


# Process Frame Pipeline


# Main Code
